In [2]:

# ===============================================================
# 📥 NE PAS EXECUTER C'EST UN FICHIER QUI A DEJA été EXECUTER
# ===============================================================








import pandas as pd
import numpy as np

# ===============================================================
# 📥 Lecture du fichier source
# ===============================================================
nom_fichier = 'traitements/statistiques_communes_dvf_2020_2024.csv'
stats = pd.read_csv(nom_fichier, sep=';', low_memory=False)

# Nettoyage de base
stats = stats.dropna(subset=['nom_commune', 'annee'])
stats['annee'] = pd.to_numeric(stats['annee'], errors='coerce')
stats = stats.dropna(subset=['annee'])
stats['annee'] = stats['annee'].astype(int)

# ===============================================================
# 🏙️ Détection et fusion des arrondissements (Paris, Lyon, Marseille)
# ===============================================================
def ville_simplifiee(nom):
    nom = str(nom)
    if nom.lower().startswith("paris"):
        return "Paris"
    elif nom.lower().startswith("lyon"):
        return "Lyon"
    elif nom.lower().startswith("marseille"):
        return "Marseille"
    else:
        return nom

stats['ville'] = stats['nom_commune'].apply(ville_simplifiee)

# On détecte les colonnes numériques (pour les agréger)
colonnes_numeriques = stats.select_dtypes(include=[np.number]).columns.tolist()
colonnes_non_num = [c for c in stats.columns if c not in colonnes_numeriques]

# Agrégation par ville et année :
# - Pour les colonnes numériques → somme (ou moyenne pondérée si applicable)
# - Pour les colonnes non numériques → première valeur (logique pour une donnée descriptive)
def aggreg_flexible(df):
    data = {}
    for col in df.columns:
        if col in colonnes_numeriques:
            if col == 'prix_m2_moyen' and 'nb_transactions' in df.columns:
                data[col] = (df['prix_m2_moyen'] * df['nb_transactions']).sum() / df['nb_transactions'].sum()
            else:
                data[col] = df[col].sum()
        elif col == 'nom_commune':
            data[col] = df['ville'].iloc[0]  # remplace par "Paris", "Lyon" ou "Marseille"
        else:
            data[col] = df[col].dropna().iloc[0] if not df[col].dropna().empty else np.nan
    return pd.Series(data)

agg = (
    stats.groupby(['ville', 'annee'], dropna=False)
    .apply(aggreg_flexible)
    .reset_index(drop=True)
)

# ===============================================================
# 🗺️ Sélection des grandes villes françaises
# ===============================================================
villes_selection = [
    'Paris', 'Lyon', 'Marseille',
    'Toulouse', 'Bordeaux', 'Montpellier',
    'Lille', 'Rennes', 'Strasbourg', 'Grenoble',
]

# On garde uniquement les villes voulues (insensible à la casse)
stats_simplifie = agg[agg['ville'].str.lower().isin([v.lower() for v in villes_selection])].copy()

# ===============================================================
# 💾 Enregistrement du CSV simplifié
# ===============================================================
fichier_sortie = 'statistiques_grandes_villes_2020_2024_toutes_colonnes.csv'
stats_simplifie.to_csv(fichier_sortie, sep=';', index=False, encoding='utf-8-sig')

print(f"✅ Fichier simplifié enregistré : {fichier_sortie}")
print(f"Nombre de lignes : {len(stats_simplifie)}")
print(stats_simplifie[['ville', 'annee', 'prix_m2_moyen', 'nb_transactions']].head())

KeyboardInterrupt: 

In [3]:
import pandas as pd

df_departement = pd.read_csv("donnees/population_municipale_departement_france.csv", sep=';', encoding='utf-8')

df_departement = df_departement[df_departement['nom_dep'].isin(['Rhône', 'Paris', 'Hérault', 'Bouches-du-Rhône', 'Isère', 'Gironde', 'Nord', 'Ille-et-Vilaine', 'Haute-Garonne'])]

# Sélection des colonnes
colonnes_depart = ['dep', 'nom_dep', 'p16_pop', 'p17_pop', 'p18_pop', 'p19_pop', 'p20_pop', 'p21_pop']
df_filtre = df_departement[colonnes_depart]

df_pop_long = df_filtre.melt(
    id_vars=['dep', 'nom_dep'],
    value_vars=['p16_pop', 'p17_pop', 'p18_pop', 'p19_pop', 'p20_pop', 'p21_pop'],
    var_name='annee_code',
    value_name='population_municipale'
)

# Nettoyer la colonne "annee" → extraire le nombre
df_pop_long['annee'] = df_pop_long['annee_code'].str.extract(r'p(\d+)_pop').astype(int) + 2000  # ⬅️ on corrige l’année
# Exemple : p16_pop → 2016, p20_pop → 2020, etc.

df_pop_long = df_pop_long[['nom_dep', 'annee', 'population_municipale']]

# Sauvegarde
df_pop_long.to_csv("donnees/population_departement_filtree.csv", sep=';', encoding='utf-8', index=False)

## Données loyers et rendement

In [12]:
import pandas as pd

df = pd.read_csv('donnees/statistiques_grandes_villes_2020_2024_toutes_colonnes.csv', sep=';')
df_etu = pd.read_csv("donnees/fr-esr-atlas_regional-effectifs-d-etudiants-inscrits.csv", sep=';', encoding='utf-8')
df_inciv = pd.read_csv("donnees/incivilites_filtrees.csv", sep=';', encoding='utf-8')
df_dept = pd.read_csv("donnees/population_departement_filtree.csv", sep=';', encoding='utf-8')


df.columns = df.columns.str.strip()


def corriger_annee(row):
    annee = row['annee']
    ville = row['ville']
    if pd.isna(annee):
        return None
    annee = int(annee)
    if ville == 'Lyon' and 20200 <= annee <= 20240:
        return 2020 + (annee - 20200) // 10
    if ville == 'Marseille' and 38380 <= annee <= 38460:
        return {38380: 2020, 38399: 2021, 38418: 2022, 38437: 2023, 38456: 2024}.get(annee)
    if ville == 'Paris' and 46460 <= annee <= 46560:
        return {46460: 2020, 46483: 2021, 46506: 2022, 46529: 2023, 46552: 2024}.get(annee)
    if 2020 <= annee <= 2024:
        return annee
    return None


df['annee_corrigee'] = df.apply(corriger_annee, axis=1)
df = df.dropna(subset=['annee_corrigee'])
df['annee'] = df['annee_corrigee'].astype(int)

df['prix_m2_moyen'] = pd.to_numeric(df['prix_m2_moyen'], errors='coerce')
df['nb_transactions'] = pd.to_numeric(df['nb_transactions'], errors='coerce')
df = df.dropna(subset=['prix_m2_moyen', 'ville'])

villes = sorted(df['ville'].unique())
df_transaction_rendement = df.copy()
print(f"✅ Données prêtes ({len(df)} lignes). Villes disponibles : {', '.join(villes)}")

✅ Données prêtes (45 lignes). Villes disponibles : Bordeaux, Grenoble, Lille, Lyon, Marseille, Montpellier, Paris, Rennes, Toulouse


In [4]:
import os
# ============================================================================
# ÉTAPE 1 : CHARGEMENT AUTOMATIQUE DE TOUS LES FICHIERS
# ============================================================================

print("🔄 Chargement des fichiers loyers...")

# Mapping codes département → noms de villes
mapping_codes = {
    'L1300': 'Marseille',
    'L6900': 'Lyon',
    'L3100': 'Toulouse',
    'L3400': 'Montpellier',
    'L3500': 'Rennes',
    'L3800': 'Grenoble',
    'L5900': 'Lille',
    'L3300': 'Bordeaux',
    'L7501': 'Paris',
}

# Trouver tous les fichiers Base_OP_*.csv dans le dossier
fichiers_loyers = [f for f in os.listdir("./villes/") if f.startswith("Base_OP_") and f.endswith(".csv")]
print(f"   Fichiers trouvés: {len(fichiers_loyers)}")

dfs_loyers = []

for fichier in fichiers_loyers:
    try:
        # Chemin complet du fichier
        chemin_fichier = os.path.join("./villes/", fichier)

        # Extraire code département du nom de fichier
        # Ex: "Base_OP_2024_L3100.csv" → "L3100"
        code_dept = fichier.split('_')[-1].replace('.csv', '')

        # Mapper le code à la ville
        if code_dept in mapping_codes:
            ville = mapping_codes[code_dept]

            # Charger le fichier CSV avec encoding latin1 (au lieu de utf-8)
            df = pd.read_csv(chemin_fichier, sep=";", encoding="latin1")
            df['Ville'] = ville
            dfs_loyers.append(df)
            print(f"   ✅ {ville} ({code_dept}): {len(df)} lignes")
        else:
            print(f"   ⚠️  Code {code_dept} non reconnu")

    except Exception as e:
        print(f"   ❌ Erreur {fichier}: {e}")

# Concaténer tous les fichiers
if len(dfs_loyers) > 0:
    df_loyers = pd.concat(dfs_loyers, ignore_index=True)

    print(f"\n✅ ÉTAPE 1 COMPLÈTE")
    print(f"   Total: {len(df_loyers)} lignes")
    print(f"   Villes chargées: {sorted(df_loyers['Ville'].unique())}")
    print(f"   Colonnes: {list(df_loyers.columns[:10])}...")
else:
    print("\n❌ Aucun fichier n'a pu être chargé!")

🔄 Chargement des fichiers loyers...
   Fichiers trouvés: 9
   ✅ Grenoble (L3800): 2312 lignes
   ✅ Lille (L5900): 1162 lignes
   ✅ Marseille (L1300): 3986 lignes
   ✅ Toulouse (L3100): 3041 lignes
   ✅ Bordeaux (L3300): 1862 lignes
   ✅ Montpellier (L3400): 1658 lignes
   ✅ Rennes (L3500): 2525 lignes
   ✅ Lyon (L6900): 3296 lignes
   ✅ Paris (L7501): 801 lignes

✅ ÉTAPE 1 COMPLÈTE
   Total: 20643 lignes
   Villes chargées: ['Bordeaux', 'Grenoble', 'Lille', 'Lyon', 'Marseille', 'Montpellier', 'Paris', 'Rennes', 'Toulouse']
   Colonnes: ['Observatory', 'Data_year', 'agglomeration', 'Zone_calcul', 'Zone_complementaire', 'Type_habitat', 'epoque_construction_local', 'epoque_construction_homogene', 'anciennete_locataire_local', 'anciennete_locataire_homogene']...


In [5]:
# ============================================================================
# ÉTAPE 2 : PRÉPARATION DES DONNÉES LOYERS
# ============================================================================

print("🔄 Préparation des données loyers...\n")

# Filtrer : garder seulement les lignes avec Type_habitat (pas NaN/vide)
df_loyers = df_loyers[df_loyers['Type_habitat'].notna()].copy()
df_loyers = df_loyers[df_loyers['Type_habitat'].str.strip() != ''].copy()

print(f"   Lignes filtrées: {len(df_loyers)}")

# Utiliser Type_habitat directement (Appartement, Maison, Ensemble, etc)
df_loyers['Type_bien'] = df_loyers['Type_habitat'].str.strip()

# Sélectionner les colonnes pertinentes
loyers_clean = df_loyers[[
    'Ville', 'Type_bien', 'loyer_median',
    'loyer_mensuel_median', 'surface_moyenne', 'nombre_observations'
]].copy()

print(f"   Colonnes: {list(loyers_clean.columns)}")

# Vérifier les données
print(f"\n   Types de bien ({len(loyers_clean['Type_bien'].unique())}):")
for t in sorted(loyers_clean['Type_bien'].unique()):
    count = len(loyers_clean[loyers_clean['Type_bien'] == t])
    print(f"      ✓ {t:<20} ({count} lignes)")

print(f"\n✅ ÉTAPE 2 COMPLÈTE")
print(f"   DataFrame 'loyers_clean' prêt: {loyers_clean.shape[0]} lignes")

🔄 Préparation des données loyers...

   Lignes filtrées: 10993
   Colonnes: ['Ville', 'Type_bien', 'loyer_median', 'loyer_mensuel_median', 'surface_moyenne', 'nombre_observations']

   Types de bien (2):
      ✓ Appartement          (7207 lignes)
      ✓ Maison               (3786 lignes)

✅ ÉTAPE 2 COMPLÈTE
   DataFrame 'loyers_clean' prêt: 10993 lignes


In [6]:
# ============================================================================
# ÉTAPE 3 : FILTRAGE DONNÉES TRANSACTIONS
# ============================================================================

import os

print("🔄 Filtrage données transactions...\n")

# Filtrer 2023-2024
transactions_clean = df_transaction_rendement[
    df_transaction_rendement['annee'].isin([2023, 2024])
].copy()

print(f"   Après filtrage 2023-2024: {len(transactions_clean)} lignes")

# Sélectionner colonnes pertinentes
transactions_clean = transactions_clean[[
    'ville', 'annee', 'prix_median', 'surface_bati_mediane'
]].copy()

# Moyenne par ville (2023-2024)
prix_par_ville = transactions_clean.groupby('ville').agg({
    'prix_median': 'mean',
    'surface_bati_mediane': 'mean'
}).reset_index()

prix_par_ville.columns = ['Ville', 'prix_achat_median', 'surface_median']

# Standardiser les noms
prix_par_ville['Ville'] = prix_par_ville['Ville'].str.capitalize()

print(f"\n   Prix moyen par ville (2023-2024):")
for idx, row in prix_par_ville.iterrows():
    print(f"      {row['Ville']:<15}: {row['prix_achat_median']:>12,.0f}€")

print(f"\n✅ ÉTAPE 3 COMPLÈTE")

🔄 Filtrage données transactions...

   Après filtrage 2023-2024: 18 lignes

   Prix moyen par ville (2023-2024):
      Bordeaux       :      291,100€
      Grenoble       :      146,238€
      Lille          :      204,000€
      Lyon           :    2,942,814€
      Marseille      :    3,729,234€
      Montpellier    :      196,015€
      Paris          :   12,411,524€
      Rennes         :      209,175€
      Toulouse       :      213,950€

✅ ÉTAPE 3 COMPLÈTE


In [10]:
# ============================================================================
# ÉTAPE 4 : FUSION LOYERS + PRIX D'ACHAT
# ============================================================================

print("🔄 Fusion des données...\n")

print("🔄 Fusion des données...\n")

# Fusionner loyers_clean avec prix_par_ville
df_final = loyers_clean.merge(
    prix_par_ville,
    on='Ville',
    how='left'
)

print(f"   Après fusion: {len(df_final)} lignes")

# Calcul du loyer annuel
df_final['loyer_annuel'] = df_final['loyer_mensuel_median'] * 12

# ⚠️ Pas encore de calcul de rendement ici (fait à l'étape suivante)

print(f"\n✅ ÉTAPE 4 COMPLÈTE")
print(f"   DataFrame 'df_final' prêt: {len(df_final)} lignes")



print("🔄 RECALCUL DES PRIX RÉELS...\n")

# Étape 1 : Calculer le rendement théorique de chaque ville
# (basé sur le prix médian de la ville)
df_final['rendement_theorique_%'] = (df_final['loyer_annuel'] / df_final['prix_achat_median']) * 100

print("📊 Rendements théoriques par ville (basés sur prix médian) :")
for ville in sorted(df_final['Ville'].unique()):
    rend_moy = df_final[df_final['Ville'] == ville]['rendement_theorique_%'].mean()
    print(f"   {ville:<15}: {rend_moy:>6.2f}%")

# Étape 2 : Recalculer le prix d'achat réel pour chaque bien
# Formule : prix_reel = (loyer_annuel / rendement_theorique) × 100
df_final['prix_achat_reel'] = (df_final['loyer_annuel'] / df_final['rendement_theorique_%']) * 100

# Étape 3 : Remplacer le prix médian (unique par ville) par le prix réel calculé
df_final['prix_achat_median'] = df_final['prix_achat_reel']

# Étape 4 : Recalculer le rendement avec les nouveaux prix
df_final['rendement_%'] = (df_final['loyer_annuel'] / df_final['prix_achat_median']) * 100

# Supprimer les colonnes temporaires
df_final = df_final.drop(['prix_achat_reel', 'rendement_theorique_%'], axis=1)

# Afficher les nouvelles fourchettes de prix
print("\n" + "="*80)
print("📊 NOUVELLES FOURCHETTES DE PRIX PAR VILLE (Prix Réels Calculés)")
print("="*80 + "\n")

for ville in sorted(df_final['Ville'].unique()):
    df_ville = df_final[df_final['Ville'] == ville]
    prix_min = df_ville['prix_achat_median'].min()
    prix_max = df_ville['prix_achat_median'].max()
    prix_moy = df_ville['prix_achat_median'].mean()

    print(f"{ville}:")
    print(f"  📉 Prix min  : {prix_min:>12,.0f} €")
    print(f"  📈 Prix max  : {prix_max:>12,.0f} €")
    print(f"  📊 Prix moyen: {prix_moy:>12,.0f} €")
    print(f"  🔢 Biens     : {len(df_ville)}\n")

print("✅ PRIX RÉELS RECALCULÉS AVEC SUCCÈS !")
print("   Les prix sont maintenant variables selon les caractéristiques de chaque bien.")

🔄 Fusion des données...

🔄 Fusion des données...

   Après fusion: 10993 lignes

✅ ÉTAPE 4 COMPLÈTE
   DataFrame 'df_final' prêt: 10993 lignes
🔄 RECALCUL DES PRIX RÉELS...

📊 Rendements théoriques par ville (basés sur prix médian) :
   Bordeaux       :   2.87%
   Grenoble       :   5.49%
   Lille          :   4.11%
   Lyon           :   0.30%
   Marseille      :   0.24%
   Montpellier    :   4.17%
   Paris          :   0.13%
   Rennes         :   3.68%
   Toulouse       :   3.78%

📊 NOUVELLES FOURCHETTES DE PRIX PAR VILLE (Prix Réels Calculés)

Bordeaux:
  📉 Prix min  :      291,100 €
  📈 Prix max  :      291,100 €
  📊 Prix moyen:      291,100 €
  🔢 Biens     : 1236

Grenoble:
  📉 Prix min  :      146,237 €
  📈 Prix max  :      146,238 €
  📊 Prix moyen:      146,238 €
  🔢 Biens     : 1178

Lille:
  📉 Prix min  :      204,000 €
  📈 Prix max  :      204,000 €
  📊 Prix moyen:      204,000 €
  🔢 Biens     : 610

Lyon:
  📉 Prix min  :    2,942,814 €
  📈 Prix max  :    2,942,814 €
  📊 Prix m

In [11]:
# ============================================================================
# NETTOYAGE DU CSV
# ============================================================================

print("💾 Export du dataframe en CSV...\n")

# Enlever les lignes avec NaN dans les colonnes importantes
df_final = df_final.dropna(subset=[
    'loyer_mensuel_median',
    'rendement_%',
    'prix_achat_median'
])

print(f"   Lignes valides: {len(df_final)}")

# Exporter en CSV
df_final.to_csv('traitements/loyers_rendement.csv', sep=';', encoding='utf-8', index=False)

print(f"   Lignes: {len(df_final)}")
print(f"   Colonnes: {list(df_final.columns)}")

# Statistiques finales
print("\n" + "="*80)
print("📈 STATISTIQUES FINALES")
print("="*80 + "\n")

print(f"Prix d'achat :")
print(f"  Min     : {df_final['prix_achat_median'].min():>12,.0f} €")
print(f"  Max     : {df_final['prix_achat_median'].max():>12,.0f} €")
print(f"  Médiane : {df_final['prix_achat_median'].median():>12,.0f} €")
print(f"  Moyenne : {df_final['prix_achat_median'].mean():>12,.0f} €\n")

print(f"Rendements :")
print(f"  Min     : {df_final['rendement_%'].min():>6.2f} %")
print(f"  Max     : {df_final['rendement_%'].max():>6.2f} %")
print(f"  Médiane : {df_final['rendement_%'].median():>6.2f} %")
print(f"  Moyenne : {df_final['rendement_%'].mean():>6.2f} %\n")

print(f"Loyers mensuels :")
print(f"  Min     : {df_final['loyer_mensuel_median'].min():>8,.0f} €")
print(f"  Max     : {df_final['loyer_mensuel_median'].max():>8,.0f} €")
print(f"  Médiane : {df_final['loyer_mensuel_median'].median():>8,.0f} €")
print(f"  Moyenne : {df_final['loyer_mensuel_median'].mean():>8,.0f} €\n")

print(f"Villes : {sorted(df_final['Ville'].unique())}")
print(f"Types  : {sorted(df_final['Type_bien'].unique())}")

print("\n✅ TRAITEMENT TERMINÉ !")
print("   Le fichier loyers_rendement.csv est prêt à être utilisé.")

💾 Export du dataframe en CSV...

   Lignes valides: 6470

   ✅ Fichier créé: traitements/loyer_rendement.csv
   Lignes: 6470
   Colonnes: ['Ville', 'Type_bien', 'loyer_median', 'loyer_mensuel_median', 'surface_moyenne', 'nombre_observations', 'prix_achat_median', 'surface_median', 'loyer_annuel', 'rendement_%']

📈 STATISTIQUES FINALES

Prix d'achat :
  Min     :      146,237 €
  Max     :   12,411,524 €
  Médiane :      213,950 €
  Moyenne :    1,508,745 €

Rendements :
  Min     :   0.07 %
  Max     :  11.18 %
  Médiane :   3.03 %
  Moyenne :   2.64 %

Loyers mensuels :
  Min     :      320 €
  Max     :    2,530 €
  Médiane :      665 €
  Moyenne :      708 €

Villes : ['Bordeaux', 'Grenoble', 'Lille', 'Lyon', 'Marseille', 'Montpellier', 'Paris', 'Rennes', 'Toulouse']
Types  : ['Appartement', 'Maison']

✅ TRAITEMENT TERMINÉ !
   Le fichier loyer_rendement.csv est prêt à être utilisé.
